In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP

/content/gdrive/MyDrive/RecipeGen/RecipeGeneratorNLP


In [3]:
!ls
from tqdm import tqdm
import string

layer1.json	     README.md	      RecipeGenerator.xcodeproj
LICENSE		     Recipe	      RecipeTraining.ipynb
Preprocessing.ipynb  RecipeGenerator  test_df.csv


In [4]:
import re
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [5]:
import json
with open('layer1.json') as json_file:
    data = json.load(json_file)

train_data, test_data = train_test_split(data, test_size = 0.3, shuffle = True)

In [6]:
recipes = data
recipes[0]['title']

'Worlds Best Mac and Cheese'

In [7]:
def get_ingredients(recipe):    
    ingredients = recipe['ingredients']
    measurements = ['oz', 'ounces','ounce', 'cups', 'cup', 'teaspoon', '/', 'to', 'lbs','lb', 'tsp', 'jar']
    item_list = []
    for item in ingredients:
        item = item['text']
        item = item.lower()
    #     item = item.split('(')[0].strip() # remove items in brackets
        item = item.translate(str.maketrans('', '', string.punctuation))
        item = re.sub(r'\s*\([^)]*\)', '', item)
        item = ''.join([char for char in item if not char.isdigit()]) # remove numbers
        for measurement in measurements:
            item = item.replace(measurement, '').strip() # remove measurements
        item.strip()
        item = re.sub('\s\s+', ' ', item)
        if item:
            item_list.append(item)

    ingredient_list = ' </I> '.join(item_list)
    return ingredient_list

In [8]:
get_ingredients(recipes[3])

'milk </I> water </I> butter </I> mashed potaes box homestyle </I> can whole kernel corn drained </I> cheddar cheese </I> frenchfried onions'

In [9]:
def get_instructions(recipe):
    instructions = recipe['instructions']
    instr_str = ""
    for instr in instructions:
      recipe = re.sub(r"\s", " ", instr['text'])
      recipe = re.sub(r'[^\w\s]', ' ', recipe)
      recipe = re.sub('\s\s+', ' ', recipe)
      instr_str += " " + recipe.lower().strip() + "."
    return instr_str

get_instructions(recipes[32])

' for the cupcakes preheat the oven to 350 degrees f fill the cupcake pan with paper liners. beat the granulated sugar and butter until light and fluffy in a standing mixer. beat in the eggs. add the lemon zest and juice and beat until combined. sift the flour baking powder baking soda and salt in a separate bowl. alternately add the buttermilk and flour mixture to the standing mixer beating well between each addition. fill the cupcake liners two thirds full. bake the cupcakes until a toothpick inserted comes out clean 18 to 22 minutes. refrigerate the cupcakes until they are completely cooled. for the filling cook 1 3 cup water 1 cup of the blueberries and granulated sugar over medium heat for 10 minutes stirring frequently. once the blueberries begin to burst add the remaining 1 cup blueberries and cook on medium heat 8 to 10 minutes. the filling is done when it coats the back of a metal spoon. for the frosting beat the butter and cream cheese until light and fluffy in a standing mix

In [10]:
df = pd.DataFrame()
recipes[0].keys()

dict_keys(['ingredients', 'url', 'partition', 'title', 'id', 'instructions'])

In [11]:
def build_text_files(recipes):
  in_recipes=[]
  for recipe in tqdm(recipes):
    ingredients = get_ingredients(recipe)
    instr = get_instructions(recipe)
    qna = { "title": recipe['title'],
            "ingred": ingredients,
            "instructions": instr }
    in_recipes.append(qna)
  return in_recipes

In [12]:
train_df=build_text_files(train_data)

100%|██████████| 720804/720804 [02:29<00:00, 4831.53it/s]


In [13]:
train_df=pd.DataFrame(train_df)

In [14]:
train_df.head()

,title,ingred,instructions
0,Smoked Pork Butt,pounds fresh pork butt roast </I> tablespoons ...,if desired soak the pork butt in a brine solu...
1,Low Fat Fresh Peach Cobbler,ripe peaches skinned and sliced </I> tablespoo...,cinnamon and sugar for shaking on top. combin...
2,Mom's Hawaiian Wedding Cake,cans crushed pineapple </I> angel food cake ri...,in a 8x10 pan break up the angel food cake in...
3,Zesty Chicken Breasts,s lemons zest of grated </I> boneless skinless...,rub equal amounts of lemon zest into each sid...
4,Daily Intake of Vegetables Chili,ground beef </I> oil </I> lg onion chopped </I...,brown ground beef until completely brown. rin...


In [15]:
test_df=build_text_files(test_data)

  5%|▌         | 16206/308916 [00:04<01:17, 3783.64it/s]


KeyboardInterrupt: ignored

In [ ]:
test_df=pd.DataFrame(test_df)

In [ ]:
test_df.head()

,title,ingred,instructions
0,Butternut Squash Lasagna,butternut squash diced about squash </I> tbsp ...,preheat oven to 450f. oil 2 large shallow bak...
1,Randy's Delicious Crock Pot Stuffing,butter </I> celery finely chopped </I> onion f...,for the bread around christmas thanksgiving e...
2,Wortel Salade - Carrot Salad,carrot peeled and chopped </I> celery diced </...,combine carrots celery and bell pepper. moist...
3,Good Fries,large russet potaes </I> salt </I> oil for dee...,peel the potatoes and cut into 3 8 inch 1 cm ...
4,Tiramisu,egg yolks </I> white sugar divided </I> pound ...,in a medium bowl beat together the egg yolks ...


In [16]:
train_df.to_csv('train_df.csv')

In [ ]:
test_df.to_csv('test_df.csv')